# Clothes

### Data Format: (examples, features)

* Input (60000, 28, 28, 1)
* Output (60000,)
* X_train  -- (60000,28,28, 1)
* X_test  -- (10000,28,28, 1)
* Y_train  -- (60000, 10)
* Y_test  -- (10000, 10)

### Model
* Conv > Maxpool > Conv > Maxpool > Flatten > FC > FC
* 28,28,1 > 28,28,32 > 14,14,32 > 14,14,64 > 7,7,64 > 7x7x64=3136 > 1024 > 10


### Results
* Accuracy: 86% (should increase with more epochs)

In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt

C:\Users\abido\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Loading the dataset

In [2]:
fashion_mnist = keras.datasets.fashion_mnist

(X_train, Y_train), (X_test, Y_test) = fashion_mnist.load_data()
print(X_train.shape)
print(Y_train.shape)

(60000, 28, 28)
(60000,)


### Preprocessing

In [3]:
#X_train = X_train / 255.0
#X_test = X_test / 255.0

X_train = X_train.reshape([-1, 28, 28, 1])
X_test = X_test.reshape([-1, 28, 28, 1])


def convert_to_one_hot(Y, C):
    """
    Y: (1, number of examples)
    C: number of classes
    returns: (number of classes, C)
    """
    Y = np.eye(C)[Y.reshape(-1)]
    return Y

Y_train = convert_to_one_hot(Y_train, 10)
Y_test = convert_to_one_hot(Y_test, 10)

print(X_train.shape)
print(Y_train.shape)

(60000, 28, 28, 1)
(60000, 10)


### Model

In [4]:
# Training Parameters 
learning_rate = 0.001
num_steps = 500
batch_size = 128
display_step = 10

# Network Parameters
num_input = 784 # Clothes data input (img shape: 28*28)
height = 28
width = 28
num_classes = 10 # Clothes total classes (0-9 digits)
dropout = 1.0 # Dropout, probability to keep units
channels = 1

In [5]:
# placeholders
X = tf.placeholder(tf.float32, [None, height, width, channels])  ## num_input == # of features
Y = tf.placeholder(tf.float32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32) # dropout (keep probability)

In [6]:
# parameters dicts
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'conv1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'conv2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'fc1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'fc2': tf.Variable(tf.random_normal([1024, num_classes]))
}

biases = {
    'bconv1': tf.Variable(tf.random_normal([32])),
    'bconv2': tf.Variable(tf.random_normal([64])),
    'bfc1': tf.Variable(tf.random_normal([1024])),
    'bfc2': tf.Variable(tf.random_normal([num_classes]))
}

In [7]:
# func to add conv layer
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

In [8]:
# func to add maxpool layer
def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')

In [9]:
# func to create the CNN model
def conv_net(x, weights, biases, dropout):
    # MNIST data input is a 1-D vector of 784 features (28*28 pixels)
    # Reshape to match picture format [Height x Width x Channel]
    # Tensor input become 4-D: [Batch Size, Height, Width, Channel]
    #x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution Layer + Max Pooling (down-sampling)
    conv1 = conv2d(x, weights['conv1'], biases['bconv1'])
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer + Max Pooling (down-sampling)
    conv2 = conv2d(conv1, weights['conv2'], biases['bconv2'])
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['fc1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['fc1']), biases['bfc1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    preds = tf.add(tf.matmul(fc1, weights['fc2']), biases['bfc2'])
    return preds

In [10]:
# Construct model
logits = conv_net(X, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)

In [14]:
print(logits)
print(prediction)

Tensor("Add_1:0", shape=(?, 10), dtype=float32)
Tensor("Softmax:0", shape=(?, 10), dtype=float32)


In [15]:
# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [16]:
# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [17]:
## Helper function to create minibatches for training
import math

def random_mini_batches(X, Y, mini_batch_size = 64, seed = 0):
    """
    Creates a list of random minibatches from (X, Y)
    
    Arguments:
    X -- input data, of shape (input size, number of examples) (m, Hi, Wi, Ci)
    Y -- true "label" vector (containing 0 if cat, 1 if non-cat), of shape (1, number of examples) (m, n_y)
    mini_batch_size - size of the mini-batches, integer
    seed -- this is only for the purpose of grading, so that you're "random minibatches are the same as ours.
    
    Returns:
    mini_batches -- list of synchronous (mini_batch_X, mini_batch_Y)
    """
    
    m = X.shape[0]                  # number of training examples
    mini_batches = []
    np.random.seed(seed)
    
    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[permutation,:,:,:]
    shuffled_Y = Y[permutation,:]

    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = math.floor(m/mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[k * mini_batch_size : k * mini_batch_size + mini_batch_size,:,:,:]
        mini_batch_Y = shuffled_Y[k * mini_batch_size : k * mini_batch_size + mini_batch_size,:]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[num_complete_minibatches * mini_batch_size : m,:,:,:]
        mini_batch_Y = shuffled_Y[num_complete_minibatches * mini_batch_size : m,:]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches

In [18]:
seed = 3
minibatch_size = 128
training_epochs = 5
m = X_train.shape[0]

In [19]:
# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        
        epoch_cost = 0.                       
        seed = seed + 1
        minibatches = random_mini_batches(X_train, Y_train, minibatch_size, seed)
        num_minibatches = len(minibatches)
        i = 0
        for minibatch in minibatches:
            i+=1
            (minibatch_X, minibatch_Y) = minibatch
            
            _, minibatch_cost = sess.run([optimizer, cost], feed_dict={X: minibatch_X, Y: minibatch_Y, keep_prob: dropout})
            
            epoch_cost += minibatch_cost / num_minibatches

            if i% 100 == 0:
                print(i, end = ' ')
                
        print("")
        acc = sess.run(accuracy, feed_dict={X: X_train[:3000], Y: Y_train[:3000], keep_prob: 1.0})
        print("Epoch: {} || Train Accuracy: {} || Cost {}".format(epoch, acc, epoch_cost))

    print("Optimization Finished!")
    print("Test Accuracy:", accuracy.eval({X: X_test[:3000], Y: Y_test[:3000], keep_prob:1.0}))

100 200 300 400 
Epoch: 0 || Train Accuracy: 0.8266666531562805 || Cost 697586.4103144992
100 200 300 400 
Epoch: 1 || Train Accuracy: 0.8560000061988831 || Cost 281879.00691297976
100 200 300 400 
Epoch: 2 || Train Accuracy: 0.8579999804496765 || Cost 195883.79202758532
100 200 300 400 
Epoch: 3 || Train Accuracy: 0.8830000162124634 || Cost 151441.98113506124
100 200 300 400 
Epoch: 4 || Train Accuracy: 0.8776666522026062 || Cost 113273.89399611863
Optimization Finished!
Test Accuracy: 0.8436667
